A Fórmula Mágica é um modelo de fator que classifica as ações por dois fatores:

1. O valor de uma empresa em relação aos seus lucros determina quão “barato” é o preço de mercado de uma ação. Greenblatt define “Barato” como o valor de uma empresa em relação aos seus lucros. Na maioria das vezes, podemos ver isto representado pelo P/L, Greenblatt prefere olhar para EV/EBIT. Isto permite que empresas com diferentes estruturas de endividamento e impostos sejam comparadas mais facilmente.

2. O retorno sobre o capital determina o quão “boa” é uma empresa. “Bom” é representado pelo ROIC, Greenblatt quantifica a quantidade de capital tangível necessária para operar um negócio e quanto dinheiro cada unidade de capital aplicado irá render.

In [1]:
earning_yield = "EV/EBIT"
return_on_capital = "ROIC"

In [2]:
import warnings
warnings.filterwarnings('ignore')
# from datetime import datetime, date
import numpy as np
import pandas as pd
# import yfinance as yf

def pct_to_float(number):
    """Convert string to float, remove % char and set decimal point to '.'."""
    return float(number.strip("%").replace(".", "").replace(",", "."))

dados do site https://www.fundamentus.com.br/resultado.php

In [3]:
import requests
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
url = 'https://www.fundamentus.com.br/resultado.php'
#junta com a requests
r = requests.get(url, headers=header)
# read_html do pandas põe a tabela num dataframe
funda = pd.read_html(r.text, index_col="Papel",
                     decimal=',', thousands='.',encoding='ISO-8859-1', 
                     converters={'ROE': pct_to_float,
                                 'ROIC': pct_to_float,
                                 'Div.Yield':pct_to_float,
                                 'Mrg Ebit':pct_to_float,
                                 'Mrg. Líq.':pct_to_float,
                                 'Cresc. Rec.5a':pct_to_float,
                                 },
)[0]
dfunds = pd.DataFrame(funda)
dfunds.tail(5)

,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
Papel,,,,,,,,,,,,,,,,,,,,
CEPE6,43.00,534.72,2.03,0.379,0.0,0.233,9.84,2.63,-0.38,8.68,6.48,14.4,0.07,1.1,10.01,0.38,0.0,1.580000e+09,5.15,331.55
UBBR4,7.49,610.27,1.99,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.33,0.0,1.031720e+10,0.00,10.58
UBBR11,14.75,1201.81,3.91,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.33,0.0,1.031720e+10,0.00,10.58
UBBR3,18.00,1466.61,4.77,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.33,0.0,1.031720e+10,0.00,10.58
CEPE3,128.00,1591.72,6.04,1.127,0.0,0.695,29.30,7.83,-1.13,13.88,10.37,14.4,0.07,1.1,10.01,0.38,0.0,1.580000e+09,5.15,331.55


In [4]:
dfunds.columns

Index(['Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro',
       'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit',
       'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq',
       'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

filtros e ranking

In [10]:
funds = dfunds.copy()

funds =  funds[funds[earning_yield] > 0]
funds =  funds[funds[return_on_capital] > 0]

funds =  funds[funds['Liq.2meses'] > 300000] #Volume diário médio negociado nos últimos 2 meses
funds =  funds[funds['P/L'] > 0] 
# funds =  funds[(funds['P/L'] > 0) & (funds['P/L'] < 60)] 
funds =  funds[funds['Dív.Brut/ Patrim.'] < 4]
# funds =  funds[funds['Cresc. Rec.5a'] > 0]
# funds =  funds[funds['ROE'] > 0]
# funds =  funds[funds['EV/EBITDA'] > 0]


""" magic formula rank."""
funds["Rank_earnings_yield"]   = funds[earning_yield].rank(ascending=True, method="min")
funds["Rank_return_on_capital"]= funds[return_on_capital].rank(ascending=False, method="min")
funds["Rank_Final"] = (funds["Rank_earnings_yield"] + funds["Rank_return_on_capital"])
funds.sort_values(by="Rank_Final", ascending=True, inplace=True)
funds.reset_index(inplace=True)
funds.index = funds.index + 1
# print(dfunds.head(15).to_string())
funds.head(30)

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
1,PSSA3,26.70,7.94,1.46,0.552,5.29,0.316,5.60,0.67,-2.68,...,1.09,59.64,18.33,4.202580e+07,1.185790e+10,0.00,15.92,1.0,1.0,2.0
2,PETR4,42.69,4.07,1.44,1.038,16.98,0.543,-76.70,2.31,-1.13,...,0.95,25.76,35.47,1.273520e+09,3.860070e+11,0.79,20.27,7.0,14.0,21.0
3,PETR3,44.08,4.20,1.49,1.072,16.44,0.561,-79.20,2.39,-1.17,...,0.95,25.76,35.47,4.080000e+08,3.860070e+11,0.79,20.27,8.0,14.0,22.0
4,WIZC3,6.27,10.31,2.18,0.955,7.10,0.437,-17.21,2.19,-1.21,...,0.90,24.37,21.18,2.654090e+06,4.592540e+08,0.77,11.69,5.0,17.0,22.0
5,CAMB3,11.40,6.28,2.20,1.007,2.61,1.361,5.83,4.35,13.24,...,1.93,33.41,35.03,7.200280e+05,2.192080e+08,0.13,28.07,18.0,5.0,23.0
6,VLID3,19.04,7.70,1.15,0.794,5.03,0.581,2.31,3.39,25.20,...,2.00,22.55,14.90,9.450320e+06,1.358760e+09,0.54,1.44,10.0,20.0,30.0
7,UNIP3,62.90,8.43,2.33,1.148,5.03,1.054,5.14,5.11,-9.17,...,1.92,29.80,27.61,8.689050e+05,2.808340e+09,0.47,21.79,22.0,10.0,32.0
8,CMIN3,6.18,11.00,2.96,1.954,12.57,1.144,3.80,5.79,-7.20,...,2.96,33.73,26.92,4.815200e+07,1.144420e+10,0.74,-9.63,32.0,4.0,36.0
9,VALE3,65.75,7.47,1.56,1.434,9.24,0.654,15.24,3.50,-1.79,...,1.28,20.67,20.91,1.838600e+09,1.909650e+11,0.35,7.85,11.0,25.0,36.0
10,UNIP6,66.90,8.96,2.47,1.221,5.20,1.121,5.46,5.44,-9.75,...,1.92,29.80,27.61,1.103350e+07,2.808340e+09,0.47,21.79,28.0,10.0,38.0


retira bancos

In [6]:
setor=20
url = f'http://www.fundamentus.com.br/resultado.php?setor={setor}'
hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201' ,
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01' ,
           'Accept-Encoding': 'gzip, deflate' ,
           }
content = requests.get(url, headers=hdr)
df = pd.read_html(content.text, decimal=",", thousands='.')[0]
bancos = list(df['Papel'])

funds = funds[~funds['Papel'].isin(bancos)]
funds.head(30)

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
1,PSSA3,26.70,7.94,1.46,0.552,5.29,0.316,5.60,0.67,-2.68,...,1.09,59.64,18.33,4.202580e+07,1.185790e+10,0.00,15.92,1.0,1.0,2.0
2,PETR4,42.69,4.07,1.44,1.038,16.98,0.543,-76.70,2.31,-1.13,...,0.95,25.76,35.47,1.273520e+09,3.860070e+11,0.79,20.27,7.0,14.0,21.0
3,PETR3,44.08,4.20,1.49,1.072,16.44,0.561,-79.20,2.39,-1.17,...,0.95,25.76,35.47,4.080000e+08,3.860070e+11,0.79,20.27,8.0,14.0,22.0
4,WIZC3,6.27,10.31,2.18,0.955,7.10,0.437,-17.21,2.19,-1.21,...,0.90,24.37,21.18,2.654090e+06,4.592540e+08,0.77,11.69,5.0,17.0,22.0
5,CAMB3,11.40,6.28,2.20,1.007,2.61,1.361,5.83,4.35,13.24,...,1.93,33.41,35.03,7.200280e+05,2.192080e+08,0.13,28.07,18.0,5.0,23.0
6,VLID3,19.04,7.70,1.15,0.794,5.03,0.581,2.31,3.39,25.20,...,2.00,22.55,14.90,9.450320e+06,1.358760e+09,0.54,1.44,10.0,20.0,30.0
7,UNIP3,62.90,8.43,2.33,1.148,5.03,1.054,5.14,5.11,-9.17,...,1.92,29.80,27.61,8.689050e+05,2.808340e+09,0.47,21.79,22.0,10.0,32.0
8,CMIN3,6.18,11.00,2.96,1.954,12.57,1.144,3.80,5.79,-7.20,...,2.96,33.73,26.92,4.815200e+07,1.144420e+10,0.74,-9.63,32.0,4.0,36.0
9,VALE3,65.75,7.47,1.56,1.434,9.24,0.654,15.24,3.50,-1.79,...,1.28,20.67,20.91,1.838600e+09,1.909650e+11,0.35,7.85,11.0,25.0,36.0
10,UNIP6,66.90,8.96,2.47,1.221,5.20,1.121,5.46,5.44,-9.75,...,1.92,29.80,27.61,1.103350e+07,2.808340e+09,0.47,21.79,28.0,10.0,38.0


retirar seguradoras

In [7]:
setor=31
url = f'http://www.fundamentus.com.br/resultado.php?setor={setor}'
hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201' ,
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01' ,
           'Accept-Encoding': 'gzip, deflate' ,
           }
content = requests.get(url, headers=hdr)
df = pd.read_html(content.text, decimal=",", thousands='.')[0]
seguros = list(df['Papel'])

funds = funds[~funds['Papel'].isin(seguros)]
funds.head(30)

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
2,PETR4,42.69,4.07,1.44,1.038,16.98,0.543,-76.70,2.31,-1.13,...,0.95,25.76,35.47,1.273520e+09,3.860070e+11,0.79,20.27,7.0,14.0,21.0
3,PETR3,44.08,4.20,1.49,1.072,16.44,0.561,-79.20,2.39,-1.17,...,0.95,25.76,35.47,4.080000e+08,3.860070e+11,0.79,20.27,8.0,14.0,22.0
5,CAMB3,11.40,6.28,2.20,1.007,2.61,1.361,5.83,4.35,13.24,...,1.93,33.41,35.03,7.200280e+05,2.192080e+08,0.13,28.07,18.0,5.0,23.0
6,VLID3,19.04,7.70,1.15,0.794,5.03,0.581,2.31,3.39,25.20,...,2.00,22.55,14.90,9.450320e+06,1.358760e+09,0.54,1.44,10.0,20.0,30.0
7,UNIP3,62.90,8.43,2.33,1.148,5.03,1.054,5.14,5.11,-9.17,...,1.92,29.80,27.61,8.689050e+05,2.808340e+09,0.47,21.79,22.0,10.0,32.0
8,CMIN3,6.18,11.00,2.96,1.954,12.57,1.144,3.80,5.79,-7.20,...,2.96,33.73,26.92,4.815200e+07,1.144420e+10,0.74,-9.63,32.0,4.0,36.0
9,VALE3,65.75,7.47,1.56,1.434,9.24,0.654,15.24,3.50,-1.79,...,1.28,20.67,20.91,1.838600e+09,1.909650e+11,0.35,7.85,11.0,25.0,36.0
10,UNIP6,66.90,8.96,2.47,1.221,5.20,1.121,5.46,5.44,-9.75,...,1.92,29.80,27.61,1.103350e+07,2.808340e+09,0.47,21.79,28.0,10.0,38.0
11,KEPL3,9.91,6.74,2.61,1.178,8.67,1.289,4.73,5.84,7.81,...,1.68,33.03,38.76,1.083970e+07,6.816180e+08,0.31,33.78,33.0,6.0,39.0
12,LEVE3,35.20,6.65,3.04,1.075,26.57,1.515,4.92,5.85,10.39,...,1.91,36.02,45.73,1.949340e+07,1.568530e+09,0.31,18.50,37.0,3.0,40.0


retirar BDR

In [8]:
# Padrão a ser buscado XYZW3[2,3,4,5]
padrao = r'[A-Z]{4}3[2-5]'

# Filtrando as linhas onde o padrão ocorre na coluna 'Papel'
funds = funds[~funds['Papel'].str.contains(padrao)]
funds.head(30)

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
2,PETR4,42.69,4.07,1.44,1.038,16.98,0.543,-76.70,2.31,-1.13,...,0.95,25.76,35.47,1.273520e+09,3.860070e+11,0.79,20.27,7.0,14.0,21.0
3,PETR3,44.08,4.20,1.49,1.072,16.44,0.561,-79.20,2.39,-1.17,...,0.95,25.76,35.47,4.080000e+08,3.860070e+11,0.79,20.27,8.0,14.0,22.0
5,CAMB3,11.40,6.28,2.20,1.007,2.61,1.361,5.83,4.35,13.24,...,1.93,33.41,35.03,7.200280e+05,2.192080e+08,0.13,28.07,18.0,5.0,23.0
6,VLID3,19.04,7.70,1.15,0.794,5.03,0.581,2.31,3.39,25.20,...,2.00,22.55,14.90,9.450320e+06,1.358760e+09,0.54,1.44,10.0,20.0,30.0
7,UNIP3,62.90,8.43,2.33,1.148,5.03,1.054,5.14,5.11,-9.17,...,1.92,29.80,27.61,8.689050e+05,2.808340e+09,0.47,21.79,22.0,10.0,32.0
8,CMIN3,6.18,11.00,2.96,1.954,12.57,1.144,3.80,5.79,-7.20,...,2.96,33.73,26.92,4.815200e+07,1.144420e+10,0.74,-9.63,32.0,4.0,36.0
9,VALE3,65.75,7.47,1.56,1.434,9.24,0.654,15.24,3.50,-1.79,...,1.28,20.67,20.91,1.838600e+09,1.909650e+11,0.35,7.85,11.0,25.0,36.0
10,UNIP6,66.90,8.96,2.47,1.221,5.20,1.121,5.46,5.44,-9.75,...,1.92,29.80,27.61,1.103350e+07,2.808340e+09,0.47,21.79,28.0,10.0,38.0
11,KEPL3,9.91,6.74,2.61,1.178,8.67,1.289,4.73,5.84,7.81,...,1.68,33.03,38.76,1.083970e+07,6.816180e+08,0.31,33.78,33.0,6.0,39.0
12,LEVE3,35.20,6.65,3.04,1.075,26.57,1.515,4.92,5.85,10.39,...,1.91,36.02,45.73,1.949340e+07,1.568530e+09,0.31,18.50,37.0,3.0,40.0


In [9]:
funds

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
2,PETR4,42.69,4.07,1.44,1.038,16.98,0.543,-76.70,2.31,-1.13,...,0.95,25.76,35.47,1.273520e+09,3.860070e+11,0.79,20.27,7.0,14.0,21.0
3,PETR3,44.08,4.20,1.49,1.072,16.44,0.561,-79.20,2.39,-1.17,...,0.95,25.76,35.47,4.080000e+08,3.860070e+11,0.79,20.27,8.0,14.0,22.0
5,CAMB3,11.40,6.28,2.20,1.007,2.61,1.361,5.83,4.35,13.24,...,1.93,33.41,35.03,7.200280e+05,2.192080e+08,0.13,28.07,18.0,5.0,23.0
6,VLID3,19.04,7.70,1.15,0.794,5.03,0.581,2.31,3.39,25.20,...,2.00,22.55,14.90,9.450320e+06,1.358760e+09,0.54,1.44,10.0,20.0,30.0
7,UNIP3,62.90,8.43,2.33,1.148,5.03,1.054,5.14,5.11,-9.17,...,1.92,29.80,27.61,8.689050e+05,2.808340e+09,0.47,21.79,22.0,10.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,LOGG3,20.91,11.12,0.57,9.703,3.34,0.342,5.28,13.22,-1.34,...,1.77,2.79,5.15,4.878870e+06,3.729940e+09,0.51,16.33,156.0,153.0,309.0
159,MELK3,4.77,13.60,0.84,0.974,11.76,0.404,0.74,26.95,1.22,...,3.08,1.89,6.17,6.717880e+05,1.171940e+09,0.24,21.63,155.0,159.0,314.0
160,JALL3,7.43,103.68,1.09,1.345,5.78,0.335,1.20,21.13,-1.05,...,3.88,1.94,1.05,5.014450e+06,2.072120e+09,1.94,9.80,160.0,158.0,318.0
161,EZTC3,16.75,19.67,0.80,3.505,1.22,0.634,1.76,65.52,2.58,...,5.19,1.13,4.09,2.976060e+07,4.598390e+09,0.18,10.57,161.0,161.0,322.0


A fórmula:
1. Estabeleça uma capitalização de mercado mínima.
2. Exclua ações de serviços públicos e financeiras  (ou seja, fundos mútuos, bancos e companhias de seguros).
3. Excluir empresas estrangeiras (ADRs)
4. Determine o rendimento dos lucros da empresa = EBIT/EV
5. Determine o retorno sobre o capital da empresa = EBIT/ (Ativo Fixo Líquido + Capital de Giro).
6. Classifique todas as empresas acima da capitalização de mercado escolhida pelo maior rendimento de lucros e maior retorno sobre o capital (classificado como porcentagens)
7. Invista em 20 a 30 empresas com melhor classificação, acumulando 2 a 3 posições por mês durante um período de 12 meses
8. Reequilibre o portfólio uma vez por ano, vendendo os perdedores uma semana antes do ano e os vencedores uma semana depois do ano.
9. Continuar por um período de longo prazo (5 a 10+ anos)

https://en.wikipedia.org/wiki/Magic_formula_investing

O retorno sobre o capital de Greenblatt difere de um valor típico de ROE ou ROIC. Dentro da Fórmula Mágica, o retorno sobre o capital de uma empresa é medido como EBIT/capital tangível empregado. Em outras palavras, estamos tentando encontrar os custos tangíveis para o negócio na geração dos lucros reportados dentro do período, onde o capital tangível empregado é definido mais precisamente como Capital Circulante Líquido mais Ativos Fixos Líquidos.

O capital de giro líquido é simplesmente o total do ativo circulante menos o passivo circulante, com um ajuste para remover dívidas com juros de curto prazo do passivo circulante e outro para remover o excesso de caixa. Greenblatt não oferece detalhes sobre como o excesso de caixa deve ser considerado, mas muitas vezes é calculado com base em uma porcentagem do caixa necessário em relação às vendas geradas em um período.